In [1]:
import pandas as pd
import numpy as np 
import itertools
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img 
from keras.models import Sequential 
from keras import optimizers
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense  
from keras import applications  
from keras.utils.np_utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import backend as K

import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
%matplotlib inline
import math  
import datetime
import time

Using TensorFlow backend.


In [2]:
#Default dimensions we found online
img_width, img_height = 299, 299  
   
#Create a bottleneck file
top_model_weights_path = 'bottleneck_fc_model-inception_resnet_v2.h5' 

# loading up our datasets
train_data_dir = 'storage/train'  
validation_data_dir = 'storage/valid'
test_data_dir = 'storage/test'
   
# number of epochs to train top model  
epochs = 20 #this has been changed after multiple model run  
# batch size used by flow_from_directory and predict_generator  
batch_size = 50

In [3]:
inception_resnet_v2 = InceptionResNetV2(include_top=False, weights='imagenet')

In [4]:
datagen = ImageDataGenerator(rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)  

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

start = datetime.datetime.now()

generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

nb_train_samples = len(generator.filenames)
num_classes = len(generator.class_indices)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))

bottleneck_features_train = inception_resnet_v2.predict_generator(generator, predict_size_train)

np.save('bottleneck_features_train-inception_resnet_v2.npy', bottleneck_features_train)

end= datetime.datetime.now() 
elapsed= end-start 
print ('Time: ', elapsed)

Found 3867 images belonging to 6 classes.


/home/evan13/miniconda3/envs/tensorflow/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:700: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/evan13/miniconda3/envs/tensorflow/lib/python3.6/site-packages/keras_preprocessing/image/image_data_generator.py:708: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Time:  0:13:10.549699


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

start = datetime.datetime.now() 

datagen = ImageDataGenerator(rescale=1. / 255)

generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

nb_validation_samples = len(generator.filenames)

predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))

bottleneck_features_validation = inception_resnet_v2.predict_generator(
generator, predict_size_validation)

np.save('bottleneck_features_validation-inception_resnet_v2.npy', bottleneck_features_validation)

end= datetime.datetime.now() 
elapsed= end-start 
print ('Time: ', elapsed)

Found 1030 images belonging to 6 classes.
Time:  0:03:31.567432


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

start = datetime.datetime.now()

datagen = ImageDataGenerator(rescale=1. / 255)

generator = datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

nb_test_samples = len(generator.filenames)

predict_size_test = int(math.ceil(nb_test_samples / batch_size))

bottleneck_features_test = inception_resnet_v2.predict_generator(
generator, predict_size_test)

np.save('bottleneck_features_test-inception_resnet_v2.npy', bottleneck_features_test) 

end= datetime.datetime.now() 
elapsed= end-start 
print ('Time: ', elapsed)

Found 1018 images belonging to 6 classes.


/home/evan13/miniconda3/envs/tensorflow/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 33554432 bytes but only got 0. Skipping tag 516
  " Skipping tag %s" % (size, len(data), tag))
/home/evan13/miniconda3/envs/tensorflow/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 69337600 bytes but only got 0. Skipping tag 3072
  " Skipping tag %s" % (size, len(data), tag))
/home/evan13/miniconda3/envs/tensorflow/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 19988480 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/home/evan13/miniconda3/envs/tensorflow/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2046820352 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag

Time:  0:03:23.493019


In [ ]:
datagen_top = ImageDataGenerator(rescale=1./255)

In [ ]:
#training data
generator_top = datagen_top.flow_from_directory(  
         train_data_dir,  
         target_size=(img_width, img_height),  
         batch_size=batch_size,  
         class_mode='categorical',  
         shuffle=False)  
   
nb_train_samples = len(generator_top.filenames)  
num_classes = len(generator_top.class_indices)  
   
# load the bottleneck features saved earlier  
train_data = np.load('bottleneck_features_train-inception_resnet_v2.npy')  
   
# get the class lebels for the training data, in the original order  
train_labels = generator_top.classes  
   
# convert the training labels to categorical vectors  
train_labels = to_categorical(train_labels, num_classes=num_classes)

Found 3867 images belonging to 6 classes.


In [ ]:
#validation data
generator_top = datagen_top.flow_from_directory(  
         validation_data_dir,  
         target_size=(img_width, img_height),  
         batch_size=batch_size,  
         class_mode=None,  
         shuffle=False)  
   
nb_validation_samples = len(generator_top.filenames)  
   
validation_data = np.load('bottleneck_features_validation-inception_resnet_v2.npy')  
   

validation_labels = generator_top.classes  
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

Found 1030 images belonging to 6 classes.


In [ ]:
#testing data
generator_top = datagen_top.flow_from_directory(  
         test_data_dir,  
         target_size=(img_width, img_height),  
         batch_size=batch_size,  
         class_mode=None,  
         shuffle=False)  
   
nb_test_samples = len(generator_top.filenames)  
   
test_data = np.load('bottleneck_features_test-inception_resnet_v2.npy')  
   

test_labels = generator_top.classes  
test_labels = to_categorical(test_labels, num_classes=num_classes)

Found 1018 images belonging to 6 classes.


In [ ]:

#This is the best model we found. For additional models, check out I_notebook.ipynb
start = datetime.datetime.now()
model = Sequential()

model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(500, activation=keras.layers.LeakyReLU(alpha=0.3)))
model.add(Dropout(0.5)) 
model.add(Dense(100, activation=keras.layers.LeakyReLU(alpha=0.3)))  
model.add(Dropout(0.5))  
model.add(Dense(50, activation=keras.layers.LeakyReLU(alpha=0.3)))  
model.add(Dropout(0.3)) 
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])  

history = model.fit(train_data, train_labels,
                    epochs=100,
                    batch_size=batch_size,
                    validation_data=(validation_data, validation_labels))  

model.save_weights(top_model_weights_path)  

(eval_loss, eval_accuracy) = model.evaluate(  
 validation_data, validation_labels, batch_size=batch_size, verbose=1)

print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))  
print("[INFO] Loss: {}".format(eval_loss))  
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

/home/evan13/miniconda3/envs/tensorflow/lib/python3.6/site-packages/keras/activations.py:211: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Train on 3867 samples, validate on 1030 samples
Epoch 1/100
3867/3867 [==============================] - 39s 10ms/step - loss: 4.2522 - acc: 0.3486 - val_loss: 1.5807 - val_acc: 0.6243
Epoch 2/100
3867/3867 [==============================] - 23s 6ms/step - loss: 2.5429 - acc: 0.5268 - val_loss: 1.2693 - val_acc: 0.7204
Epoch 3/100
3867/3867 [==============================] - 23s 6ms/step - loss: 2.4344 - acc: 0.5627 - val_loss: 1.0375 - val_acc: 0.7796
Epoch 4/100
3867/3867 [==============================] - 23s 6ms/step - loss: 2.2033 - acc: 0.6160 - val_loss: 1.2666 - val_acc: 0.7534
Epoch 5/100
3867/3867 [==============================] - 33s 8ms/step - loss: 2.0120 - acc: 0.6475 - val_loss: 1.1541 - val_acc: 0.7825
Epoch 6/100
3867/3867 [==============================] - 34s 9ms/step - loss: 1.8769 - acc: 0.6822 - val_loss: 1.0731 - val_acc: 0.7883
Epoch 7/100
3867/3867 [==============================] - 34s 9ms/step - loss: 1.6667 - acc: 0.7148 - val_loss: 1.0904 - val_acc: 0.8010

In [ ]:
model.summary()

In [ ]:

#Graphing our training and validation
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Inception-ResNet V2 Training and validation accuracy')
plt.ylabel('accuracy')  
plt.xlabel('epoch')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Inception-ResNet V2 Training and validation loss')
plt.ylabel('loss')  
plt.xlabel('epoch')
plt.legend()
plt.show()